# Olympic Cycling Medals by Country
## 1896-2012
### Data from [Kaggle](https://www.kaggle.com/the-guardian/olympic-games).

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

First, we want to read in our csv and display the head to make sure everything looks ok.

In [ ]:
SUMMER = pd.read_csv('summer.csv')

In [ ]:
SUMMER.head(10)

Get medal count by country for cycling and sort descending.

In [ ]:
cycling = SUMMER.query('Sport == "Cycling"').groupby(['Country'], as_index=False)['Medal'].count().sort_values(by='Medal', ascending=False)

In [ ]:
cycling.head(10)

Let's put this data on a map!

In [ ]:
layout = dict(title=dict(text=('Olympic Cycling Medals Awarded by Country '
                             '1896-2012'),
                             x=0.5,
                             ),
              geo=dict(countrycolor='#444444',
                       showcountries=True,
                       oceancolor='deepskyblue',
                       showocean=True,
                       lakecolor='deepskyblue',
                       showlakes=True,
                       projection={'type': 'natural earth'},
                       )
              )

In [ ]:
data = go.Choropleth(locations=cycling['Country'],
                     locationmode='ISO-3',
                     z=cycling['Medal'],
                     colorscale=px.colors.sequential.Oryel)

In [ ]:
fig = go.Figure(data=data, layout=layout)
fig.show()

We can see that this doesn't look quite right. In Europe, Germany and the Netherlands, which are in the Top-10 from our cycling.head(10) selection above, don't reflect any medals on our map. There is an issue with the country codes! Using ISO-3, we need to replace some country codes in order to have them display correctly on our map.

In [ ]:
cycling['Country'] = cycling['Country'].replace(
        ['GER', 'NED', 'DEN', 'SUI', 'RSA', 'GRE', 'LAT', 'POR', 'URU'],
        ['DEU', 'NLD', 'DNK', 'CHE', 'ZAF', 'GRC', 'LVA', 'PRT', 'URY']
        )

In [ ]:
data = go.Choropleth(locations=cycling['Country'],
                     locationmode='ISO-3',
                     z=cycling['Medal'],
                     colorscale=px.colors.sequential.Oryel)

In [ ]:
fig = go.Figure(data=data, layout=layout)
fig.show()

Now, that looks better!

In reading the plotly documentation, I discovered the scope arg in layout.geo. This gave me the idea to create a region selector for the final figure, to allow for an easy zoom-in to any continent.

In [ ]:
def region_scopes(region: str):
    # Pass to geo for region selection on map.
    return dict(countrycolor='#444444',
                showcountries=True,
                oceancolor='deepskyblue',
                showocean=True,
                lakecolor='deepskyblue',
                showlakes=True,
                projection={'type': 'natural earth'},
                scope=region,
                )

Add region selector and display figure.

In [ ]:
fig.update_layout(
        updatemenus=[
            dict(buttons=list([
                dict(label='World',
                     method='relayout',
                     args=['geo', region_scopes('world')]),
                dict(label='Africa',
                     method='relayout',
                     args=['geo', region_scopes('africa')]),
                dict(label='Asia',
                     method='relayout',
                     args=['geo', region_scopes('asia')]),
                dict(label='Europe',
                     method='relayout',
                     args=['geo', region_scopes('europe')]),
                dict(label='North America',
                     method='relayout',
                     args=['geo', region_scopes('north america')]),
                dict(label='South America',
                     method='relayout',
                     args=['geo', region_scopes('south america')])
            ]),
             x=0.2,
             y=1.15,
             direction='right'
            )
        ],
    )